In [85]:
import csv
import pickle
from datetime import date, datetime, timedelta
import numpy as np

# ToDo: move all methods to a separate file and import them

def convert_txt_to_dict(filename, path_to_save):
    content = []

    with open(filename, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            content.append(row)

    data_ftse_100_dict = {}
    tickers_list = [ticker[:-2] for ticker in content[0][1:]]

    for row in content[1:]:
        data_ftse_100_dict[row[0]] = {}
        for i, ticker in enumerate(tickers_list):
            data_ftse_100_dict[row[0]][ticker] = row[i + 1]

    pickle.dump(data_ftse_100_dict, open(path_to_save, "wb"))

    
txt_filename = 'ftse_100_time_data.csv'
path_to_save = "./data_ftse_100_dict_04032017.p"

convert_txt_to_dict(txt_filename, path_to_save)

In [73]:
data_ftse_100_dict['1990-07-11']

{'AAL': '',
 'ABF': '362.56',
 'ADM': '',
 'ADN': '',
 'AGK': '',
 'AHT': '197.584',
 'ANTO': '600.21',
 'ARM': '',
 'AV': '461.259',
 'AZN': '',
 'BA': '8640.0',
 'BAB': '1.2690000000000001',
 'BARC': '6000.64',
 'BATS': '',
 'BDEV': '157.30100000000002',
 'BG': '42.995',
 'BLND': '260.501',
 'BLT': '',
 'BNZL': '89.0',
 'BP': '321.0',
 'BRBY': '',
 'BT-A': '311.0',
 'CCH': '',
 'CCL': '',
 'CNA': '',
 'CPG': '',
 'CPI': '172.003',
 'CRH': '',
 'DC': '',
 'DGE': '675.937',
 'DLG': '',
 'EXPN': '',
 'EZJ': '',
 'FRES': '',
 'GFS': '',
 'GKN': '1053.642',
 'GLEN': '',
 'GSK': '3408.0',
 'HIK': '',
 'HL': '',
 'HMSO': '476.407',
 'HSBA': '37.472',
 'IAG': '',
 'IHG': '',
 'III': '',
 'IMT': '',
 'INTU': '',
 'ITRK': '',
 'ITV': '',
 'JMAT': '271.842',
 'KGF': '475.96',
 'LAND': '462.10699999999997',
 'LGEN': '420.1',
 'LLOY': '',
 'LSE': '',
 'MGGT': '59.141000000000005',
 'MKS': '188.635',
 'MNDI': '',
 'MRW': '185.0',
 'NG': '',
 'NXT': '49.0',
 'OML': '',
 'PRU': '205.261',
 'PSN': '1

How to write `add_dates` method
http://stackoverflow.com/questions/3240458/how-to-increment-the-day-in-datetime-python

In [74]:
def datetime_to_date(datetime_date):
    datetime_date.date().year
    str_date = '{}-{}-{}'.format(datetime_date.date().strftime('%Y'), datetime_date.date().strftime('%m'), datetime_date.date().strftime('%d'))
    return str_date
    
def date_to_datetime(date):
    return datetime.strptime(date, '%Y-%m-%d')

def find_start_end_dates(ticker, dates_dict):
    # this method looks when a company started to be in ftse 100. Assume that some dates may not have dictionaries
    # with ticker. No ticker in keys.
    start = ""
    end = ""
    for date, stock_values in dates_dict.items():
        if ticker in stock_values.keys():
            if stock_values[ticker]:
                # check if start is empty
                if not start:
                    start = date
                else:
                    end = date
    
    return start, end # dates in datetime format

def get_dates_range(start_date, end_date):
    # assume start_date and end_date are in datetime format
    dates_range = []
    delta = end_date - start_date
    date_index = start_date
    for i in range(delta.days + 1):
        dates_range.append(date_index)
        date_index += timedelta(days=1)
    return dates_range

def add_interpolated_values(dates_range, ticker, dates_dict):
    modified_dates_dict = dates_dict
    interpolated_value = (modified_dates_dict[datetime_to_date(start_indicator)] + modified_dates_dict[datetime_to_date(end_indicator)]) / 2
    for date in dates_range:
        # Initialise new date
        if datetime_to_date(date) not in modified_dates_dict.keys():
            modified_dates_dict[datetime_to_date(date)] = {}
        modified_dates_dict[datetime_to_date(date)][ticker] = interpolated_value
    # return modified_dates_dict add average values using start_indicator and end_indicator
    
def add_missing_date(missing_date, dates_dict, ticker):
    complete_dates_dict = dates_dict
    if missing_date not in dates_dict.keys():
        complete_dates_dict[missing_date] = {}
        complete_dates_dict[missing_date][ticker] = ""
    return complete_dates_dict

def interpolate(dates_dict, ticker, start_date, end_date):
    # assume start_date and end_date are in datetime format
    dates_range = get_dates_range(start_date, end_date)
    print(len(dates_range))
    print(dates_range[-2])
    start_indicator = dates_range[0]
    end_indicator = dates_range[0]
    for date in dates_range[1:]:
        dates_dict = add_missing_date(datetime_to_date(date), dates_dict, ticker)
        if not dates_dict[datetime_to_date(date)][ticker]:
            end_indicator = date
        else:
            if (start_indicator - end_indicator).days > 1:
                dates_dict = add_interpolated_values(dates_range[1:-1], ticker, dates_dict)
            else:
                start_indicator = end_indicator
    return dates_dict

In [75]:
start, end = find_start_end_dates("SAB", data_ftse_100_dict)
print(end, start)
data_ftse_100_dict_interp = interpolate(data_ftse_100_dict, ticker, date_to_datetime(start), date_to_datetime(end))

2015-04-29 1999-03-08
5897
2015-04-28 00:00:00


In [82]:
tickers_list = data_ftse_100_dict_interp['1990-07-11'].keys()
date_point = '1999-07-11'
valid_tickers = []
for ticker in tickers_list:
    start, end = find_start_end_dates(ticker, data_ftse_100_dict)
    if date_to_datetime(date_point) > date_to_datetime(start):
        valid_tickers.append(ticker)

print(len(valid_tickers), len(tickers_list))

72 100


'03'

## Calculate correlations

In [83]:
def get_components_distance(time_window, time_start, time_end, ticker_1, ticker_2, dates_dict):
    components_distance = {}
    dates_range = get_dates_range(time_start, time_end)
    
    for date in dates_range:
        s_vector_1 = get_s_vector(time_window, ticker_1, dates_dict, date)
        s_vector_2 = get_s_vector(time_window, ticker_2, dates_dict, date)
        components_distance[datetime_to_date(date)] = (np.dot(s_vector_1, s_vector_2) / len(s_vector_1) - np.mean(s_vector_1) * np.mean(s_vector_2)) / \
            np.sqrt((np.mean([x * x for x in s_vector_1]) - np.mean(s_vector_1) * np.mean(s_vector_1)) * \
            (np.mean([x * x for x in s_vector_1]) - np.mean(s_vector_1) * np.mean(s_vector_1)))
        
    
    return components_distance # dict where each key is time and value is distance

def get_s_vector(time_window, ticker, dates_dict, initial_time):
    return s_vector

def get_s(time, ticker, dates_dict):
    return s



In [86]:
np.dot([2,3,4], [5,2,1])

20

In [89]:
[x * x for x in [2,3,4]]

[4, 9, 16]